<a href="https://colab.research.google.com/github/yuuu76/ODL-assignment/blob/main/DNN(WQ).ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
import os
import cv2
import numpy as np
from PIL import Image
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
from sklearn.model_selection import train_test_split
import xml.etree.ElementTree as ET # for parsing XML
from PIL import Image # to read images

from tensorflow import keras
from keras.models import Sequential
from keras.layers import Conv2D,MaxPooling2D,Dense,Flatten,Dropout
from keras.layers import BatchNormalization
from keras.utils import load_img
import tensorflow as tf
from tensorflow.keras import layers, models
from keras.layers import Activation, Dropout, Flatten, Dense, GlobalMaxPooling2D, Conv2D, MaxPooling2D
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Flatten, Dropout, Activation

print("Loaded all libraries")

Loaded all libraries


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
# for label
all_images = os.listdir('/content/drive/MyDrive/ODLAssignment/cropped_images')
# for cropped and resized image
file_path = '/content/drive/MyDrive/ODLAssignment/resized_images_array.npy'
resized_images_array = np.load(file_path)

# Extract breed and dog information from filenames
breed_dog_info = [img.split('_') for img in all_images]

# Join the parts to get the full breed name
breeds = ['_'.join(parts) for parts in breed_dog_info]

# Convert lists to NumPy arrays
resized_images_array = np.array(resized_images_array)
breeds = np.array(breeds)

# Print the length of the arrays
print(f"Number of images: {len(resized_images_array)}")
print(f"Number of breeds: {len(breeds)}")


# Split the data into training and testing sets
train_images, test_images, train_labels, test_labels = train_test_split(resized_images_array, breeds, test_size=0.2, random_state=42)

# Print the shape of the resulting arrays
print(f"Shape of train_images: {train_images.shape}")
print(f"Shape of test_images: {test_images.shape}")
print(f"Shape of train_labels: {train_labels.shape}")
print(f"Shape of test_labels: {test_labels.shape}")

Number of images: 20580
Number of breeds: 20580
Shape of train_images: (16464, 250, 250, 3)
Shape of test_images: (4116, 250, 250, 3)
Shape of train_labels: (16464,)
Shape of test_labels: (4116,)


In [5]:
# Normalize pixel values to be between 0 and 1
train_images_normalized = train_images / 255.0
test_images_normalized = test_images / 255.0


In [6]:
from tensorflow.keras.models import load_model
model_path = '/content/drive/MyDrive/ODLAssignment/dnn_model.h5'
loaded_model = load_model(model_path)

## models

In [5]:
model = Sequential()
model.add(Flatten(input_shape=(250, 250, 3)))
model.add(Dense(256))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(128))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(64))
model.add(Activation('relu'))
model.add(Dropout(0.5))
model.add(Dense(1))
model.add(Activation('sigmoid'))

model.compile(optimizer='rmsprop',
              loss='binary_crossentropy',
              metrics=['accuracy'])

In [6]:
model_path = '/content/drive/MyDrive/ODLAssignment/dnn_model.h5'

# Save the model
model.save(model_path)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


In [6]:


# Define the model architecture
model = models.Sequential([
    layers.Flatten(input_shape=(250, 250, 3)),  # Assuming images are resized to 250x250 pixels
    layers.Dense(64, activation='relu'),
    layers.Dropout(0.5),
    layers.Dense(len(np.unique(train_labels)), activation='softmax')  # Output layer with softmax activation for classification
])

# Compile the model
model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])





In [ ]:
# Train the model with batch processing
batch_size = 16
history = loaded_model.fit(train_images_normalized, train_labels, epochs=10, validation_split=0.2, batch_size=batch_size, verbose = 1)



In [ ]:
# Evaluate the model on the test set
test_loss, test_accuracy = model.evaluate(test_images_normalized, test_labels)

print(f"Test Accuracy: {test_accuracy}")

# Display training history (optional)
plt.plot(history.history['accuracy'], label='accuracy')
plt.plot(history.history['val_accuracy'], label='val_accuracy')
plt.xlabel('Epoch')
plt.ylabel('Accuracy')
plt.legend()
plt.show()